# Background Measurements

## Imports

In [1]:
# Python Imports
from os import path
from IPython.core.display import display

# 3rd Party Imports
import numpy as np
import pandas as pd
from astropy.io import fits
import pyregion as pyreg

# ESO
#import eso
from astroptical.photometry import magnitude

## Notebook Setup

### Computation Setup

In [2]:
# Numpy Warnings
_ = np.seterr(invalid='ignore')

### Directories and Files

In [21]:
# Input Directories
DS9_DIR = path.join('/Users/sunil/Documents/TestESO_137_002/ESO-137-002')

# Image Files
IMG_FNS = {
    275: path.join(DS9_DIR, '08_drc_cosmicray',  'F275W_drc_chip_cleaned.fits'),
    475: path.join(DS9_DIR, '08_drc_cosmicray',  'F475W_drc_chip_cleaned.fits'),
    814: path.join(DS9_DIR, '08_drc_cosmicray',  'F814W_drc_chip_cleaned.fits'),
    160: path.join(DS9_DIR, '03_drc_sunil',      'F160W-ES0_137_002_drz.fits')
}

# Region File Names
REG_FNS = {
    275: 'background_uvis_fk5.reg',
    475: 'background_acs_fk5.reg',
    814: 'background_acs_fk5.reg',
    160: 'background_IR_fk5.reg',
}
REG_FNS = {key: path.join(DS9_DIR, '09_color_mag_regions', val) for key, val in REG_FNS.items()}

## Functions

In [22]:
# Root Mean Square
def rms(a, axis=None):
    return np.sqrt(np.mean(np.square(a), axis=axis))

## Load Data

### Load Zeropoints

In [23]:
zpts = pd.read_pickle(path.join('/Users/sunil/Documents/TestESO_137_002\
/ESO-137-002/12_vorbin/pickle_table', 'eso_002_zero_pt_table.p'))

### Load Images

In [24]:
# Instantiate
imgs, hdrs = {}, {}

# Read
for key, fileName in IMG_FNS.items():
    with fits.open(fileName) as hduList:
        
        # Get Data
        hdrs[key] = hduList[1].header
        imgs[key] = hduList[1].data

## Load Regions

In [25]:
# Instantiate
msks = {}

# Load
for key, fileName in REG_FNS.items():
    
    # Load
    msks[key] = pyreg.open(fileName).get_mask(header=hdrs[key], shape=imgs[key].shape)

## Calculate the Data

In [26]:
# Instantiate
stats = pd.DataFrame(columns=['Mean', 'Std', 'RMS'], index=imgs.keys(), dtype=float)

# Loop to Measure
for key in imgs:
    
    # Extract
    img, msk = imgs[key], msks[key]
    
    # Get the Masked Img
    img = img[msk]
    
    # Get the Stats
    stats['Mean'][key] = img.mean()
    stats['Std'][key]  = img.std()
    stats['RMS'][key]  = rms(img)

In [27]:
# Get the Threshold Flux
stats['Thresh Flux (Std)'] = 3*stats['Std']
stats['Thresh Flux (RMS)'] = 3*stats['RMS']

In [28]:
# Get the Threshold Mags
stats['Thresh Mag (Std)'], _ = magnitude(stats['Thresh Flux (Std)'], zpts['abZpt'], zpts['GalExt'])
stats['Thresh Mag (RMS)'], _ = magnitude(stats['Thresh Flux (RMS)'], zpts['abZpt'], zpts['GalExt'])

In [29]:
# Display Results
display(stats)
for key, val in stats['Thresh Mag (Std)'].items():
    print('{:.1f} mag for F{:d}W'.format(val, key))

,Mean,Std,RMS,Thresh Flux (Std),Thresh Flux (RMS),Thresh Mag (Std),Thresh Mag (RMS)
275,0.000026,0.000964,0.000964,0.002892,0.002893,29.422203,29.421805
475,-0.001262,0.002336,0.002655,0.007007,0.007964,30.792242,30.653112
814,-0.003158,0.004955,0.005875,0.014864,0.017626,30.212685,30.027642
160,-0.006965,0.002752,0.007489,0.008257,0.022466,31.052274,29.965485


29.4 mag for F275W
30.8 mag for F475W
30.2 mag for F814W
31.1 mag for F160W
